In [1]:
!pip install map-boxes

You should consider upgrading via the 'C:\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import random

c:\Python39\lib\site-packages\Cython\Compiler\Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: c:\Python39\lib\site-packages\map_boxes\compute_overlap.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [3]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...


In [ ]:
!pip install -r yolov5/requirements.txt

In [7]:
BASE_DIR = "D:\\_hack\\2021-22\\d-hacks-ai.ru\\758258"
train_images_dir = os.path.join(BASE_DIR, "train", "images")
train_labels_dir = os.path.join(BASE_DIR, "train", "labels")
test_images_dir = os.path.join(BASE_DIR, "test", "images")

In [8]:
len(os.listdir(test_images_dir))

855

In [9]:
len(os.listdir(train_images_dir))

2002

In [10]:
#распределение объектов в train
pd.Series([file_name.split('_')[-1] for file_name in os.listdir(train_labels_dir)]).value_counts()

head..txt        1741
human..txt       1740
car..txt         1647
face..txt         630
carplate..txt     246
dtype: int64

In [11]:
#создадим обучающий датасет в формате yolo
!mkdir data_for_yolo
!mkdir data_for_yolo/data
!mkdir data_for_yolo/data/images
!mkdir data_for_yolo/data/labels
!mkdir data_for_yolo/data/images/train
!mkdir data_for_yolo/data/labels/train
!mkdir data_for_yolo/data/images/test
!mkdir data_for_yolo/data/labels/test

�訡�� � ᨭ⠪�� �������.
�訡�� � ᨭ⠪�� �������.
�訡�� � ᨭ⠪�� �������.
�訡�� � ᨭ⠪�� �������.
�訡�� � ᨭ⠪�� �������.
�訡�� � ᨭ⠪�� �������.
�訡�� � ᨭ⠪�� �������.


In [ ]:
#в папке data_for_yolo создадим dataset.yaml со следующим содержимым

yaml_content = """
train: /content/data_for_yolo/data/images/train/
val: /content/data_for_yolo/data/images/test/

# number of classes
nc: 1

# class names
names: ['human']
"""

In [ ]:
human_labels_only = [file_name for file_name in os.listdir('participants/train/labels') if 'human..txt' in file_name]
print(len(human_labels_only))

1355


In [ ]:
human_images_names = [file_name.split('_human..txt')[0] for file_name in human_labels_only]
print(len(human_images_names))

1355


In [ ]:
train_images_names, test_images_names = train_test_split(human_images_names, test_size=0.2, random_state=22)
print(len(train_images_names), len(test_images_names))

1084 271


In [ ]:
#train
for train_image in train_images_names:
  #labels
  for file in os.listdir('participants/train/labels'): 
    if '_human..txt' in file and train_image in file: 
      shutil.copy('participants/train/labels/' + file, '/content/data_for_yolo/data/labels/train/' + file.replace('_human..txt', '.txt'))
  #images 
  for file in os.listdir('participants/train/images'): 
    if train_image + '.jpg' == file: 
      shutil.copy('participants/train/images/' + file, '/content/data_for_yolo/data/images/train/')

#test
for test_image in test_images_names:
  #labels
  for file in os.listdir('participants/train/labels'): 
    if '_human..txt' in file and test_image in file: 
      shutil.copy('participants/train/labels/' + file, '/content/data_for_yolo/data/labels/test/' + file.replace('_human..txt', '.txt'))
  #images 
  for file in os.listdir('participants/train/images'): 
    if test_image + '.jpg' == file: 
      shutil.copy('participants/train/images/' + file, '/content/data_for_yolo/data/images/test/')

In [ ]:
#валидация 
print(len(os.listdir('/content/data_for_yolo/data/images/test')))
print(len(os.listdir('/content/data_for_yolo/data/images/train')))

271
1084


In [ ]:
print(len(os.listdir('/content/data_for_yolo/data/labels/test')))
print(len(os.listdir('/content/data_for_yolo/data/labels/train')))

271
1084


In [ ]:
#обучение

In [ ]:
path_to_data = '/content/data_for_yolo/dataset.yaml'

In [12]:
!nvidia-smi

"nvidia-smi" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [ ]:
!python ./yolov5/train.py --img 416 --batch 16 --epochs 100 --data {path_to_data} --cfg ./yolov5/models/yolov5s.yaml --name yolov5s_results --cache

train: weights=yolov5/yolov5s.pt, cfg=./yolov5/models/yolov5s.yaml, data=/content/data_for_yolo/dataset.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-307-g92e47b8 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, o

In [ ]:
!python3 yolov5/detect.py --weights yolov5/runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.4 --source /content/data_for_yolo/data/images/test --save-txt --save-conf

detect: weights=['yolov5/runs/train/yolov5s_results/weights/best.pt'], source=/content/data_for_yolo/data/images/test, data=yolov5/data/coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-307-g92e47b8 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/271 /content/data_for_yolo/data/images/test/23-11-2021_02-19-35_PM.jpg: 256x416 2 humans, Done. (0.012s)
image 2/271 /content/data_for_yolo/data/images/test/23-11-2021_02-20-28_PM.jpg: 256x416 1 human, Done. (0.009s)
image 3/271 /content/data_for_yolo/data/images/test/23-11-2021_02-21-06_PM.jpg: 256

In [ ]:
def get_soliton_labels_df(path_to_txt_folder):
  simple_solution = []
  for detection_file in os.listdir(path_to_txt_folder):
    img_name = detection_file.split('.')[0] + '.jpg'
    with open(path_to_txt_folder + detection_file, 'r') as f:
      data = f.read()
      data = [i for i in data.split('\n') if i != '']
    for line in data:
      val = [float(i) for i in line.split()]
      cls, xywh, conf = val[0], val[1:5], val[5]
      center_x, center_y, width, height = xywh
      xmin = center_x - (width / 2)
      xmax = center_x + (width / 2)
      ymin = center_y - (height / 2)
      ymax = center_y + (height / 2)
      simple_solution.append([img_name, cls, conf, xmin, xmax, ymin, ymax])
  return simple_solution

In [ ]:
simple_solution = get_soliton_labels_df('yolov5/runs/detect/exp/labels/')
simple_solution = pd.DataFrame(simple_solution, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])

In [ ]:
def get_test_labels_df(path_to_txt_folder):
  simple_solution = []
  for detection_file in os.listdir(path_to_txt_folder):
    img_name = detection_file.split('.')[0] + '.jpg'
    with open(path_to_txt_folder + detection_file, 'r') as f:
      data = f.read()
      data = [i for i in data.split('\n') if i != '']
    for line in data:
      val = [float(i) for i in line.split()]
      cls, center_x, center_y, width, height = val
      xmin = center_x - (width / 2)
      xmax = center_x + (width / 2)
      ymin = center_y - (height / 2)
      ymax = center_y + (height / 2)
      simple_solution.append([img_name, cls, xmin, xmax, ymin, ymax])
  return simple_solution

In [ ]:
test_labels = get_test_labels_df('/content/data_for_yolo/data/labels/test/')
test_labels = pd.DataFrame(test_labels, columns=['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax'])

In [ ]:
mean_ap, average_precisions = mean_average_precision_for_boxes(test_labels, simple_solution, iou_threshold=0.5, verbose=False)

In [ ]:
mean_ap

0.5318470438910017

In [ ]:
#/content/participants/test/images

In [ ]:
!python3 yolov5/detect.py --weights yolov5/runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.4 --source /content/participants/test/images --save-txt --save-conf

detect: weights=['yolov5/runs/train/yolov5s_results/weights/best.pt'], source=/content/participants/test/images, data=yolov5/data/coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-307-g92e47b8 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/855 /content/participants/test/images/0_23-11-2021_02-18-50_PM.jpg: 288x416 Done. (0.012s)
image 2/855 /content/participants/test/images/0_23-11-2021_02-18-56_PM.jpg: 352x416 Done. (0.013s)
image 3/855 /content/participants/test/images/0_23-11-2021_02-19-19_PM.jpg: 256x416 Done. (0.012s)
image 4/855 /cont

In [ ]:
def get_soliton_labels_df_simple_solution(path_to_txt_folder):
  simple_solution = []
  for detection_file in os.listdir(path_to_txt_folder):
    img_name = detection_file.split('.')[0] + '.jpg'
    with open(path_to_txt_folder + detection_file, 'r') as f:
      data = f.read()
      data = [i for i in data.split('\n') if i != '']
    for line in data:
      val = [float(i) for i in line.split()]
      _, xywh, conf = val[0], val[1:5], val[5]
      center_x, center_y, width, height = xywh
      xmin = center_x - (width / 2)
      xmax = center_x + (width / 2)
      ymin = center_y - (height / 2)
      ymax = center_y + (height / 2)
      simple_solution.append([img_name, 3, conf, xmin, xmax, ymin, ymax])
  return simple_solution

In [ ]:
simple_solution = get_soliton_labels_df_simple_solution('yolov5/runs/detect/exp2/labels/')
simple_solution = pd.DataFrame(simple_solution, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])

In [ ]:
simple_solution.to_csv("sample_solution.csv", sep=';', index=False)

In [ ]:
df = pd.read_csv("sample_solution.csv", sep=';', index_col=None)

In [ ]:
df

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
0,24-11-2021_05-31-51_AM.jpg,3,0.825893,0.035938,0.053646,0.305556,0.373148
1,25-11-2021_06-10-03_AM.jpg,3,0.917864,0.718750,0.763392,0.169737,0.357895
2,24-11-2021_09-05-00_AM.jpg,3,0.536900,0.571484,0.587500,0.320139,0.389583
3,24-11-2021_09-05-00_AM.jpg,3,0.712740,0.873438,0.892578,0.313195,0.398611
4,24-11-2021_09-05-00_AM.jpg,3,0.770197,0.895703,0.915235,0.325695,0.408333
...,...,...,...,...,...,...,...
1421,24-11-2021_07-36-12_AM.jpg,3,0.463904,0.663021,0.692187,0.467592,0.575925
1422,24-11-2021_07-36-12_AM.jpg,3,0.824089,0.844792,0.856250,0.284259,0.345371
1423,24-11-2021_07-36-12_AM.jpg,3,0.897071,0.755729,0.769791,0.333333,0.404629
1424,23-11-2021_02-55-37_PM.jpg,3,0.888559,0.030878,0.042411,0.409869,0.468421
